## Libraries

In [1]:
import os
import rioxarray as rxr
from shapely.geometry import mapping
import geopandas as gpd
import rasterio as rio

Example:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas


In [7]:
# Set main directory and paths
main_dir = 'path_to_main_directory'
monthly_SMA = os.path.join(main_dir, 'SMA_monthly')
Clipped_path = os.path.join(main_dir, 'Clipped_SMA')
Boundary = os.path.join(main_dir, 'Boundary')
# Load example raster to get CRS
Copernicus_ex = rxr.open_rasterio(os.path.join(main_dir, 'example_copernicus_fapar.tif'), masked=True)
Copernicus_ex_crs = Copernicus_ex.rio.crs

In [8]:
# Load the shapefile
def load_shape_file(filepath):
    """Loads the shapefile to create an AOI mask for clipping."""
    shpfile = gpd.read_file(filepath)
    print("Shapefile loaded. To prepare for masking, run the function `select_shape`.")
    return shpfile

# Create AOI mask from shapefile
def select_shape(shpfile):
    col_code = 'ISO3_CODE'
    country_codes = ['ZAF', 'LSO', 'SWZ']
    selected_rows = shpfile[shpfile[col_code].isin(country_codes)]
    unioned_polygon = selected_rows.geometry.unary_union
    mask_polygon = gpd.GeoDataFrame(geometry=[unioned_polygon])
    print("Mask created.")
    return mask_polygon

shpfile = load_shape_file(os.path.join(Boundary, 'boundary_shapefile.shp'))
AOI = select_shape(shpfile)
# Assign CRS to AOI based on example raster
AOI.crs = Copernicus_ex_crs

Shapefile loaded. To prepare for masking, run the function `select_shape`.
Mask created.


In [14]:
# Process each SMA image in the monthly SMA directory
for image in os.listdir(monthly_SMA):
    # Path to current image
    path = os.path.join(monthly_SMA, image)
    # Open image as xarray
    SMA = rxr.open_rasterio(path, masked=True)
    # Clip to AOI
    clipped = SMA.rio.clip(AOI.geometry.apply(mapping),
                           crs=AOI.crs,
                           all_touched=True,
                           from_disk=True).squeeze()
    # Save clipped image with month and year in name
    clipped.rio.to_raster(os.path.join(Clipped_path, image[12:18] + '.tif'))